In [1]:
import tensorflow as tf
import numpy as np

# 学習済みネットワークで物体認識

In [2]:
# 学習済みネットワークを読み込み
f = open("classify_image_graph_def.pb", "rb")
graph_def = tf.GraphDef()
graph_def.ParseFromString(f.read())
tf.import_graph_def(graph_def, name='')
f.close()

In [3]:
# 出力のindexと名前の対応ファイル読み込み
f = open("index2name.txt", "r")
index2name = {}
for line in  f.readlines():
    index, name = line.strip().split("\t")
    index2name[int(index)] = name
f.close()

In [4]:
# 認識する画像を読み込み
f = open("images/panda.jpg", "rb")
img = f.read()
f.close()

In [5]:
# 画像認識
with tf.Session() as sess:
    softmax_tensor = sess.graph.get_tensor_by_name("softmax:0")
    feed_dict = {"DecodeJpeg/contents:0": img}
    predictions = sess.run(softmax_tensor, feed_dict=feed_dict)[0]

In [6]:
# 認識結果表示
top_n =  np.argsort( predictions )[::-1]

for i in range(5):
    idx = top_n[i]
    print(  "第%d位：%s (score=%lf)" % (i+1,index2name[idx],  predictions[idx] ) )    

第1位：giant panda, panda, panda bear, coon bear, Ailuropoda melanoleuca (score=0.891073)
第2位：indri, indris, Indri indri, Indri brevicaudatus (score=0.007791)
第3位：lesser panda, red panda, panda, bear cat, cat bear, Ailurus fulgens (score=0.002959)
第4位：custard apple (score=0.001466)
第5位：earthstar (score=0.001174)


# 学習済みネットワークとSVMで学習と認識

In [7]:
# 特徴抽出をしてSVMを学習
images = []

# 学習用
for i in range(10):
    f = open("images/%03d.png" % i, "rb")
    img = f.read()
    images.append(img)
    f.close()
labels = np.loadtxt("images/labels.txt")

In [8]:
# 学習済みネットワークから特徴抽出
features = []
with tf.Session() as sess:
    for img in images:
        pool = sess.graph.get_tensor_by_name("pool_3:0")
        feed_dict = {"DecodeJpeg/contents:0": img}
        feat = sess.run(pool, feed_dict=feed_dict)[0].flatten()
        features.append(feat)

In [9]:
# SVM学習
from sklearn.svm import SVC

svc = SVC()
svc.fit(features, labels)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [10]:
# 未学習画像を認識
test_images = []

# 学習用
for i in range(6):
    f = open("images/test%03d.png" % i, "rb")
    img = f.read()
    test_images.append(img)
    f.close()

In [11]:
# 学習済みネットワークから特徴抽出
test_features = []
with tf.Session() as sess:
    for img in test_images:
        pool = sess.graph.get_tensor_by_name("pool_3:0")
        feed_dict = {"DecodeJpeg/contents:0": img}
        feat = sess.run(pool, feed_dict=feed_dict)[0].flatten()
        test_features.append(feat)
        
# SVMで分類
prediction = svc.predict(test_features)

print("分類結果：", prediction)

分類結果： [0. 0. 0. 1. 1. 1.]
